In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import networkx as nx
import freeman as fm

In [2]:
g = fm.load('marvel.gml')
g.label_nodes('name')
g.set_all_nodes(size=15, labpos='hover')
g.set_all_edges(color=(0, 0, 0, 0.5))
g.draw()

In [3]:
bc = nx.betweenness_centrality(g)

In [4]:
import pandas as pd
data = pd.DataFrame({
    'id' : [i for i in g.nodes],
    'Name': [g.nodes[i]['name'] for i in g.nodes],
    'ConnAmnt': [g.nodes[i]['amount'] for i in g.nodes],
    'Intermediation (r)': [i for i in bc.values()],
})

In [5]:
char_infos = pd.read_csv('../dataset/characters_stats.csv', index_col=0)
semi_df = pd.merge(data, char_infos, how='inner', on=['Name']) 

In [6]:
import numpy as np
semi_df = semi_df.replace('nan', np.nan)
semi_df = semi_df[semi_df['ConnAmnt'].notna()]
semi_df['ConnAmnt'] = pd.to_numeric(semi_df['ConnAmnt'])

In [7]:
semi_df = semi_df.drop_duplicates(subset=['Name'], keep='first')

In [8]:
import statsmodels.api as sm

model = sm.OLS(semi_df['Intermediation (r)'], semi_df[['Intelligence', 'Strength', 'Power']])
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     Intermediation (r)   R-squared (uncentered):                   0.409
Model:                            OLS   Adj. R-squared (uncentered):              0.395
Method:                 Least Squares   F-statistic:                              30.43
Date:                Sat, 14 Nov 2020   Prob (F-statistic):                    5.11e-15
Time:                        18:09:24   Log-Likelihood:                          498.61
No. Observations:                 135   AIC:                                     -991.2
Df Residuals:                     132   BIC:                                     -982.5
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intelligence     0.0001   2.51e-05      4.742      0.000    6.94e-05       0.000
Strength       -1.1e-06   1.95e-05     -0.056      0.955   -3.96e-05    3.74e-05
Power        -3.899e-05   2.42e-05     -1.613      0.109   -8.68e-05    8.84e-06
==============================================================================
Omnibus:                      110.521   Durbin-Watson:                   1.055
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1090.783
Skew:                           2.870   Prob(JB):                    1.38e-237
Kurtosis:                      15.688   Cond. No.                         5.64
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""